In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import CardName
import random
import math
import time
import os
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
import re
import spacy
from typing import Callable

c:\Users\ddw\Anaconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spacy_en = spacy.load('en_core_web_sm')
spacy_zh = spacy.load("zh_core_web_sm")

def tokenizer_en(text):
    # return [tok.text for tok in spacy_en.tokenizer(text)]
    ret = []

def tokenizer_zh(text):
    # return [tok.text for tok in spacy_zh.tokenizer(text)]
    return [c for c in text]


SRC = Field(tokenize = tokenizer_en, 
                init_token = '<sos>', 
                eos_token = '<eos>', 
                lower = True)
TRG = Field(tokenize = tokenizer_zh, 
                init_token = '<sos>', 
                eos_token = '<eos>', 
                lower = True)
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}
train_data, valid_data, test_data = CardName.splits(fields=fields)

print(f'Number of train data: {len(train_data)}')
print(f'Number of valid data: {len(valid_data)}')
print(f'Number of test data: {len(test_data)}')

Number of train data: 17014
Number of valid data: 447
Number of test data: 449


In [13]:
# for data in test_data[:10]:
for data in random.sample(test_data.examples, 3):
    print(data.src, data.trg)

['angel', 'of', 'vitality'] ['活力', '天使']
['lizard', 'blades'] ['蜥蜴', '双', '刀']
['cemetery', 'gatekeeper'] ['墓地', '守门者']


In [4]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 4)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (en) vocabulary: 5604
Unique tokens in target (zh) vocabulary: 2215


In [14]:
print(*TRG.vocab.stoi.keys())

<unk> <pad> <sos> <eos> 的 之 兽 法师 妖 人 怪 地 灵 使 骑士 鬼 天使 巨人 蛇 裂片 太 兵 巨龙 魔 乙 像 手 恶魔 守护者 斗士 狼 元素 客 龙 不 武士 术士 吸血鬼 骑兵 仪式 公会 大 风暴 亚龙 信徒 大师 游魂 多 一 守卫 战士 烈焰 茜卓 专家 僧侣 先知 怒火 灵魂 神 神圣 祭师 智者 械 神秘 鱼 会 史芬斯 哨兵 暴君 能 虫 蜘蛛 心灵 树 那 艾文 邪鬼 鬼怪 鲜血 僧 咒 学者 护卫 非瑞克西亚 剑 哨卫 巨魔 月 都 队长 龙兽 力 后裔 复仇 破坏 蛮 诅咒 陷阱 食人 从 印记 地狱 狮鹫 秘耳 精怪 巨 火焰 舰队 魔力 冲锋 吞噬 指命 攫 机械 死 死亡 狂热 仙灵 伏击 传令 佣兽 精灵 肯 莉莲娜 魂 魔像 俄 命运 守护 护符 时间 残虐者 泰坦 英雄 荒野 远古 与 复生 夺命 幻象 成长 残酷 苦痛 记忆 遗迹 召唤 墙 多头龙 学徒 拒斥 永生 长 闪电 阿耶尼 食尸 黑暗 亡者 先锋 军团 劫掠者 可 向导 寇族 师 怒 战斗 无情 猫 王 精 自然 魔鬼 刃 古 基定 妮莎 导师 恶毒 意志 拉铎 明光 枯萎 殿堂 泰菲力 浪潮 独眼 祝福 突袭 行进 贝西摩斯 凤凰 墓地 家 工匠 希望 恐惧 明师 林地 洞察 混沌 灵俑 炼狱 爆发 秘教徒 米斯拉 纪念碑 统领 老兵 致命 虚空 连 迷雾 预言师 鼠 信念 冲击 击 勇士 回收 墓穴 密使 密探 幻灵 幽灵 思绪 活力 渗透者 灾祸 维多 罗堰 血 跛行 门 风 飞马 你 侍僧 反抗 地脉 岩浆 巡卫 干预 恶鬼 执法者 斥候 时 末日 杖 杰斯 狂 甲 盛宴 英勇 贤者 领主 鲁莽 加理 召现 吸血 回响 复仇者 天界 奴兽 尸嵌 巨灵 巴洛西 德鲁伊 心 攻城 斗篷 杀手 森林 死灵 法术师 波尬 狮族 猎手 眼 符文 老 联盟 觉醒 象 钢铁 饥渴 首领 骷髅 伊 佐立 启示 妖精 娜 将军 小 尸 巨兽 巫妖 指挥官 斗客 无 无畏 时刻 正义 火光 犬 玄铁 玛尔 祸害 空境 窃贼 翻腾 至 葛 蛮野 贾路 遗宝 野猪 隐士 主 冥界 利刃 力量 午夜 半 卫士 大地 女 实界 巨汉 巨海 帮 引擎 恶体 战争 探索 新手 暗夜 树灵 梦魇 欧佐夫 毁灭 深渊 炽天 焰 牛头 猎人 猎犬 疫病 盾 石 

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

tmp = next(iter(train_iterator))
print(tmp)

cpu

[torchtext.legacy.data.batch.Batch of size 32]
	[.src]:[torch.LongTensor of size 4x32]
	[.trg]:[torch.LongTensor of size 5x32]


In [6]:
from models.model4.definition import Encoder, Attention, Decoder, Seq2Seq
from models.model4.train import init_weights, train, evaluate
from utils import count_parameters, train_loop

In [7]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

model.apply(init_weights)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,406,439 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='test-card-name-model.pt', load_before_train=True)

In [8]:
from utils.translate import Translator
from models.model4.definition import beam_search
model.load_state_dict(torch.load('result/test-card-name-model.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)

In [67]:
T.translate('guardian of solitude')

([['<unk>', '守护者', '<eos>'],
  ['<unk>', '守卫', '<eos>'],
  ['<unk>', '<unk>', '<eos>']],
 [0.5978619184235284, 0.19217252471822394, 0.022983777595810998])

In [72]:
from utils import show_samples
print(f'Number of samples: {len(test_data)}')
show_samples(test_data, T, n=3, beam_size=3)

Number of samples: 449
src: [justiciar 's portal ] trg = [大司法通道]
<unk><unk>通道<eos> 	[probability: 0.27791]
<unk>的通道<eos> 	[probability: 0.05891]
<unk>法师通道<eos> 	[probability: 0.02660]

src: [kor duelist ] trg = [寇族斗客]
寇族<unk><eos> 	[probability: 0.14255]
励志<unk><eos> 	[probability: 0.10460]
励志斗客<eos> 	[probability: 0.06303]

src: [foreboding fruit ] trg = [预兆果实]
<unk><unk><eos> 	[probability: 0.79890]
<unk><unk>兽<eos> 	[probability: 0.02685]
<unk><unk><unk><eos> 	[probability: 0.02018]

